# Audit Anamoly Detection for E-commerce Orders

In [12]:
import pandas as pd

# Loading key Olist datasets
orders = pd.read_csv('olist_orders_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
product_category_translation = pd.read_csv("olist_product_category_name_translation.csv")

df = pd.merge(orders, order_items, on = 'order_id', how = "left")
df = pd.merge(df, products[["product_id", "product_category_name"]], on = "product_id", how = "left")
df = pd.merge(df, customers, on="customer_id", how="left")
df = pd.merge(df, payments.groupby("order_id").agg({"payment_value":"sum"}).reset_index(), on="order_id", how="left")
df = pd.merge(df, reviews[["order_id", "review_score", "review_comment_message"]], on="order_id", how="left")
df = pd.merge(df, sellers, on="seller_id", how="left")
df = pd.merge(df, product_category_translation, on="product_category_name", how="left")

df.to_csv("olist_datasets_merged.csv", index=False)

In [14]:
df.shape

(114092, 26)

In [15]:
# Data Preprocessing

# Load the dataset
df = pd.read_csv("olist_datasets_merged.csv")

# 1. Dropping rows with missing critical identifiers
df_cleaned = df.dropna(subset=['order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'seller_zip_code_prefix', 'seller_city', 'seller_state'])

# 2. Dropping rows with minor nulls in important operational fields
df_cleaned = df_cleaned.dropna(subset=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'payment_value', 'review_score'])

# 3. Dropping rows with missing review_comment_message
df_cleaned = df_cleaned[df_cleaned['review_comment_message'].str.strip() != '']

# 4. Filling missing product category names with 'unknown'
df_cleaned['product_category_name'] = df_cleaned['product_category_name'].fillna('unknown')
df_cleaned['product_category_name_english'] = df_cleaned['product_category_name_english'].fillna('unknown')

# Resetting index if needed
df_cleaned.reset_index(drop=True, inplace=True)

In [16]:
df_cleaned.shape

(109966, 26)

## Anamoly 1 - High freight charges relative to product price

In [18]:
df_cleaned['freight_price_anomalies'] = df_cleaned['freight_value'] > df_cleaned['price']

## Anamoly 2 - Outlier order prices

In [19]:
Q1 = df_cleaned['price'].quantile(0.25)
Q3 = df_cleaned['price'].quantile(0.75)
IQR = Q3 - Q1
df_cleaned['price_outliers'] = (df_cleaned['price'] < Q1 - 1.5 * IQR) | (df_cleaned['price'] > Q3 + 1.5 * IQR)

## Anamoly 3 - Low review scores with high prices 

In [20]:
price_75th = df_cleaned['price'].quantile(0.75)
df_cleaned['low_review_high_price'] = (df_cleaned['review_score'] <= 2) & (df_cleaned['price'] > price_75th)

## Anamoly 4 - Duplicate orders

In [21]:
df_cleaned['duplicate_orders'] = df_cleaned.duplicated(subset='order_id', keep=False)

## Anamoly 5 - Late Deliveries

In [23]:
df_cleaned['late_deliveries'] = pd.to_datetime(df_cleaned['order_delivered_customer_date']) > pd.to_datetime(df_cleaned['order_estimated_delivery_date'])

In [24]:
def classify_anomaly(row):
    if row.get('late_deliveries', False):
        return 'Late Delivery'
    elif row.get('freight_price_anomalies', False):
        return 'Freight > Price'
    elif row.get('price_outliers', False):
        return 'Price Outlier'
    elif row.get('low_review_high_price', False):
        return 'Low Review, High Price'
    elif row.get('duplicate_orders', False):
        return 'Duplicate Order ID'
    else:
        return 'No Anomaly'

df_cleaned['anomaly_type'] = df_cleaned.apply(classify_anomaly, axis=1)
df_cleaned['anomaly_type'].value_counts()

anomaly_type
No Anomaly                68632
Duplicate Order ID        19884
Late Delivery              8557
Price Outlier              7351
Freight > Price            3634
Low Review, High Price     1908
Name: count, dtype: int64

In [25]:
df_cleaned.to_csv("olist_anomalies_flagged.csv", index=False)